In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

clf = pickle.load(open("classifier.pck", "rb"))
enc = pickle.load(open("encoder.pck", "rb"))

next_matchday_df = pickle.load(open("next_matchday_df.pck", "rb"))
next_matchday_df.drop("resultClass", axis=1, inplace=True)
next_matchday_df_prep = next_matchday_df.drop(["date", "teamHomeName", "teamAwayName"], axis=1)
predictions = clf.predict(next_matchday_df_prep)
next_matchday_df["prediction"] = enc.inverse_transform(predictions)
next_matchday_df[["date", "teamHomeName", "teamAwayName", "prediction"]]

,date,teamHomeName,teamAwayName,prediction
5803,2024-05-11 15:30:00,1. FC Köln,1. FC Union Berlin,1:1
5804,2024-05-11 15:30:00,VfL Bochum,Bayer Leverkusen,0:2
5802,2024-05-11 15:30:00,Borussia Mönchengladbach,Eintracht Frankfurt,1:1
5805,2024-05-11 15:30:00,FC Augsburg,VfB Stuttgart,1:1
5800,2024-05-11 15:30:00,SC Freiburg,1. FC Heidenheim 1846,1:1
5799,2024-05-11 15:30:00,RB Leipzig,Werder Bremen,1:1
5798,2024-05-11 15:30:00,FC Bayern München,VfL Wolfsburg,2:0
5801,2024-05-11 15:30:00,1. FSV Mainz 05,Borussia Dortmund,0:1
5806,2024-05-11 15:30:00,SV Darmstadt 98,TSG 1899 Hoffenheim,1:1


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import dotenv
import os

dotenv.load_dotenv(".env")
os.getenv("EMAIL")

driver = webdriver.Chrome()
driver.get("http://kicktipp.de")

nav_button = driver.find_element(By.CLASS_NAME, "navtoggle")
nav_button.click()

email_input = driver.find_element(By.ID, "kennung")
email_input.send_keys(os.getenv("EMAIL"))
passwort_input = driver.find_element(By.ID, "passwort")
passwort_input.send_keys(os.getenv("PASSWORT"))

login_button = driver.find_element(By.NAME, "submitbutton")
login_button.click()

In [7]:
tipprunden_div = driver.find_element(By.XPATH, '//*[@id="kicktipp-content"]/div[2]/div[2]/div[1]/div/div')
for tipprunden in tipprunden_div.find_elements(By.TAG_NAME, "a"):
    if tipprunden.get_attribute('innerHTML') == os.getenv("TIPPRUNDENNAME"):
        tipprunden.click()
        break

In [8]:
cookies_div = driver.find_element(By.CLASS_NAME, "qc-cmp2-summary-buttons")
for button in cookies_div.find_elements(By.TAG_NAME, "button"):
    if button.find_element(By.TAG_NAME, "span").get_attribute("innerHTML") == "ZUSTIMMEN":
        button.click()
        break

In [9]:
driver.get(os.getenv("LINK-TIPPABGABE"))

In [10]:
from difflib import get_close_matches

tippabgabe_tabelle = driver.find_element(By.ID, "tippabgabeSpiele")
for datarow in tippabgabe_tabelle.find_elements(By.CLASS_NAME, "datarow"):
    homeTeam = datarow.find_element(By.CLASS_NAME, "col1").get_attribute("innerHTML")
    if homeTeam != "":
        selectedTeam = get_close_matches(homeTeam, next_matchday_df["teamHomeName"])[0]
        prediction = next_matchday_df[next_matchday_df["teamHomeName"] == selectedTeam]["prediction"].iloc[0].split(":")
        inputs = datarow.find_elements(By.TAG_NAME, "input")
        inputs[1].clear()
        inputs[1].send_keys(prediction[0])
        inputs[2].clear()
        inputs[2].send_keys(prediction[1])

In [11]:
driver.find_element(By.NAME, "submitbutton").click()

In [12]:
driver.close()